<a href="https://colab.research.google.com/github/Brian0718UOA/2021-Summer-Research-Scholarship/blob/main/bk_yolov5_NZTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!ls
# change directory to yolov5 folder
%cd drive/MyDrive/Colab \Notebooks/yolov5

drive  sample_data
/content/drive/MyDrive/Colab Notebooks/yolov5


In [ ]:
# do not need to clone (i.e run this cell) if yolov5 folder is uploaded to Google Colab
# !git clone https://github.com/ultralytics/yolov5.git
# %cd yolov5/
# !pip install -U -r requirements.txt

# Upload files as zip files from local machine

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import shutil
shutil.move("/content/NZTA.zip", "/content/yolov5/data")  # move NZTA.zip file into data folder

In [ ]:
%cd yolov5/data/
!unzip NZTA.zip  # zip file to folder with images

# Upload files from URL

In [7]:
import requests

url = input("paste jpg url: ")

response = requests.get(url, stream = True)

text_file = open("NZTA_cctv.jpg", "wb")
for chunk in response.iter_content(chunk_size = 1024):
  text_file.write(chunk)


text_file.close()

paste jpg url: https://www.trafficnz.info/camera/653.jpg


In [ ]:
# for saving multiple urls as jpg into destined folder so that they can be inferenced together
import requests

# save a list of urls into a list and convert the urls to jpg
url_list = [] 
url = input("input jpg url: ")
while url != "done":
  url_list.append(url)
  url = input("input jpg url or type done if no more inputs: ")
print(url_list)

for url in url_list:
  response = requests.get(url, stream = True)

  jpg_file = open('/content/sample_data/'+str(url[-7:-4])+'.jpg', 'wb')  # change path to desired  
  for chunk in response.iter_content(chunk_size = 1024):
    jpg_file.write(chunk)
  jpg_file.close()



# Inference

In [ ]:
#%cd yolov5
#%cd ../
!python detect.py --weights yolov5s.pt --img 640 --conf 0.6 --source NZTA/

In [8]:
# inferencing the image from url
!python detect.py --weights yolov5s.pt --img 640 --conf 0.2 --source NZTA_cctv.jpg

detect: weights=['yolov5s.pt'], source=NZTA_cctv.jpg, imgsz=[640, 640], conf_thres=0.2, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13

requirements: 1 package updated per /content/drive/My Drive/Colab Notebooks/yolov5/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2021-11-14 torch 1.10.0+cu111 CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
image 1/1 /content/drive/My Drive/Colab Notebooks/